In [1]:
from ipywidgets import interact, interact_manual, widgets, Label
import numpy as np
import functools
from time import time, sleep
import matplotlib.pyplot as plt
from scipy.special import comb

In [4]:
test_size_max=1024
a=np.broadcast_to(np.arange(test_size_max+1),(test_size_max+1,test_size_max+1))
b=comb(a,a.T)

nb_p=100
v_p=0.5*np.arange(1,nb_p+1,dtype=np.float64)/(nb_p+1)
v_h=-1*(np.log2(v_p)*(v_p)+np.log2(1-v_p)*(1-v_p))

def paramt1(entropy_in=0.5,test_size=512,Th_low=250,Th_high=260):
  closest_index=abs(v_h-entropy_in).argmin()
  v_proba_error=np.zeros((nb_p,),dtype=np.float64)
  for i in range(nb_p):
    v_binom=b[:,test_size]*(v_p[i]**np.arange(test_size_max+1))*((1-v_p[i])**(test_size_max-np.arange(test_size_max+1)))
    v_proba_error[i]=v_binom[:Th_low].sum()+v_binom[Th_high:].sum()
  plt.plot(v_h,v_proba_error)
  plt.plot(v_h[closest_index],v_proba_error[closest_index],'rx')

#paramt1(v_h,entropy_in=0.9,test_size=1024,Th_low=512-100,Th_high=512+100)

im=interact(paramt1)
im.widget.close()
im.widget.children[0].max=1.0
im.widget.children[0].min=0.0
im.widget.children[0].step=0.01
im.widget.children[1].max=1024
im.widget.children[1].min=8
im.widget.children[1].step=1
im.widget.children[2].max=1024
im.widget.children[2].min=0
im.widget.children[2].step=1
im.widget.children[3].max=1024
im.widget.children[3].min=0
im.widget.children[3].step=1
im.widget.children[0].description='Entropy'
im.widget.children[1].description='Latency'
im.widget.children[2].description='Thresh. Low'
im.widget.children[3].description='Thresh. High'

im.widget.children[0].layout=widgets.Layout(width='650px')
im.widget.children[1].layout=widgets.Layout(width='650px')
im.widget.children[2].layout=widgets.Layout(width='650px')
im.widget.children[3].layout=widgets.Layout(width='650px')
v_box_top=widgets.VBox([im.widget.children[0],im.widget.children[1],im.widget.children[2],im.widget.children[3],im.widget.children[4]])
display(v_box_top)

interactive(children=(FloatSlider(value=0.5, description='entropy_in', max=1.5, min=-0.5), IntSlider(value=512…